# Evaluate Supply Chain Agent
This notebook demonstrates how to:
- Evaluate the agent with Mosaic AI Agent Evaluation.

## Cluster Configuration
This notebook was tested on the following Databricks cluster configuration:
- **Databricks Runtime Version:** 16.4 LTS ML (includes Apache Spark 3.5.2, Scala 2.12)
- **Single Node** 
    - Azure: Standard_DS4_v2 (28 GB Memory, 8 Cores)
    - AWS: m5d.2xlarge (32 GB Memory, 8 Cores)

In [0]:
%pip install -U -qqqq mlflow-skinny[databricks] langgraph==0.3.4 databricks-langchain databricks-agents uv
%pip install -r ../requirements.txt --quiet
dbutils.library.restartPython()

## Load the agent

In [0]:
import mlflow
from databricks import agents

# Connect to the Unity catalog model registry
mlflow.set_registry_uri("databricks-uc")

catalog = "supply_chain_stress_test"    # Change here
schema = "agents"                       # Change here
agent_name = "supply_chain_agent"       # Change here

# Load the latest version of the model using pyfunc flavor
agent = mlflow.pyfunc.load_model(f"models:/{catalog}.{schema}.{agent_name}@latest")

## Test the agent

Interact with the agent to test its output.

In [0]:
import os
import mlflow
from dbruntime.databricks_repl_context import get_context

# TODO: set WORKSPACE_URL manually if it cannot be inferred from the current notebook
WORKSPACE_URL = None
if WORKSPACE_URL is None:
  workspace_url_hostname = get_context().browserHostName
  assert workspace_url_hostname is not None, "Unable to look up current workspace URL. This can happen if running against serverless compute. Manually set WORKSPACE_URL yourself above, or run this notebook against classic compute"
  WORKSPACE_URL = f"https://{workspace_url_hostname}"

# TODO: set secret_scope_name and secret_key_name to access your PAT
secret_scope = "ryuta"
secret_key = "token"

os.environ["HOST"] = WORKSPACE_URL
os.environ["TOKEN"] = dbutils.secrets.get(scope=secret_scope, key=secret_key)

In [0]:
from supply_chain_agent import AGENT

agent.predict({"messages": [{"role": "user", "content": "What happens if T2_4 goes down and takes 6 weeks to recover? What should I do?"}]})

## Evaluate the agent with [Agent Evaluation](https://learn.microsoft.com/azure/databricks/mlflow3/genai/eval-monitor)

You can edit the requests or expected responses in your evaluation dataset and run evaluation as you iterate your agent, leveraging mlflow to track the computed quality metrics. Evaluate your agent with one of our [predefined LLM scorers](https://learn.microsoft.com/azure/databricks/mlflow3/genai/eval-monitor/predefined-judge-scorers), or try adding [custom metrics](https://learn.microsoft.com/azure/databricks/mlflow3/genai/eval-monitor/custom-scorers).

In [0]:
import json
import mlflow.genai
from mlflow.genai.judges import custom_prompt_judge
from mlflow.entities import Trace, Feedback, SpanType
from mlflow.genai.scorers import scorer
from mlflow.genai.scorers import Guidelines, RelevanceToQuery, Safety

In [0]:
# Evaluation dataset
eval_data = [
    {
        "inputs": {
            "messages": [
                {
                    "role": "user",
                    "content": "List all downstream sites for the raw material supplied by T3_10, and include any related information about these sites."
                }
            ],
        },
        "expected_response": None
    },
    {
        "inputs": {
            "messages": [
                {
                    "role": "user",
                    "content": "Tell me the demand for T1_5 and the inventory levels of all materials needed to produce T1_5."
                }
            ],
        },
        "expected_response": None
    },
    {
        "inputs": {
            "messages": [
                {
                    "role": "user",
                    "content": "Tell me what happens if T2_8 is disrupted and requires 9 to recover. What should I do?"
                }
            ],
        },
        "expected_response": None
    },
    {
        "inputs": {
            "messages": [
                {
                    "role": "user",
                    "content": "What happens if T2_4 goes down and takes 6 weeks to recover? What should I do?"
                }
            ],
        },
        "expected_response": None
    },
    {
        "inputs": {
            "messages": [
                {
                    "role": "user",
                    "content": "There has been an incident at T3_15 and it will go down for the next 10 time units. Tell me what to do."
                }
            ],
        },
        "expected_response": None
    },
]

In [0]:
# Define evaluation scorers
scorers = [
    Guidelines(
        name="response_length",
        guidelines="The response MUST be concise and to the point not longer than 500 words.",
    ),
    Guidelines(
        name="professional_tone",
        guidelines="The response MUST be in a professional tone.",
    ),
    Guidelines(
        name="includes_next_steps",
        guidelines="The response MUST end with a specific, actionable next step that includes a concrete timeline.",
    ),
    RelevanceToQuery(),
    Safety(),
]

In [0]:
# Run evaluation
def evaluate_model(messages) -> dict:
    return agent.predict({"messages": messages})
  
results = mlflow.genai.evaluate(
    data=eval_data,
    predict_fn=evaluate_model,
    scorers=scorers
)

In [0]:
generated_traces = mlflow.search_traces(run_id=results.run_id)

tool_usage_prompt = """
Evaluate the trace and determine if the right tool was used to answer the question.

Question:
{{question}}

Tool spans to evaluate:
{{spans}}

Return ONLY a valid JSON object with exactly these keys:
- "rationale": short explanation (<=80 words)
- "result": one of [[pass]], [[partially_pass]], [[fail]]

STRICT OUTPUT RULES:
- Output a single JSON object.
- No markdown, no code fences, no extra text before/after.
- Example (copy the format exactly):
{"rationale":"...", "result":"pass"}
""".strip()

@scorer
def tool_usage(trace: Trace) -> Feedback:
    
    question = trace.search_spans(span_type=SpanType.AGENT)[0].inputs['messages'][0]['content']
    tool_spans = trace.search_spans(span_type=SpanType.TOOL)
    tool_spans_str = "\n".join(
        json.dumps(s.to_dict(), ensure_ascii=False) for s in tool_spans
    )

    judge = custom_prompt_judge(
        name="tool_usage",
        prompt_template=tool_usage_prompt,
        numeric_values={
            "pass": 1,
            "partially_pass": 0.5,
            "fail": 0,
        },
    )
    return judge(question=question, spans=tool_spans_str)

@scorer
def response_time(trace: Trace) -> Feedback:
    
    # Search particular span type from the trace
    agent_span = trace.search_spans(span_type=SpanType.AGENT)[0]

    response_time = (agent_span.end_time_ns - agent_span.start_time_ns) / 1e9 # second
    max_duration = 120
    if response_time <= max_duration:
        return Feedback(
            value="yes",
            rationale=f"Response time {response_time:.2f}s is within the {max_duration}s limit."
        )
    else:
        return Feedback(
            value="no",
            rationale=f"Response time {response_time:.2f}s exceeds the {max_duration}s limit."
        )

# Evaluate the scorer using the pre-generated traces from the prerequisite code block.
trace_check_eval_results = mlflow.genai.evaluate(
    data=generated_traces,
    scorers=[response_time, tool_usage]
)

## Assign `Production` alias to this version of the agent

In [0]:
from mlflow import MlflowClient

client = MlflowClient()
model_info = client.get_model_version_by_alias(f"{catalog}.{schema}.{agent_name}", "latest")
client.set_registered_model_alias(f"{catalog}.{schema}.{agent_name}", "production", model_info.version)

## Next steps
